# It isn't obvious from the start how to use BERT.

Hmm here's some repo for Chinese NLP models https://github.com/lonePatient/awesome-pretrained-chinese-nlp-models

Hmm let's just the pretrained one on huggingface https://huggingface.co/bert-base-chinese

In [1]:
import torch
import transformers
from transformers import BertForMaskedLM

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

PRETRAINED_MODEL_NAME = "bert-base-chinese"

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# hmmm looking at some tutorial here

Source: https://www.heywhale.com/mw/project/609f609f06b942001794ff44

- [CLS]：在做分类任务时其最后一层的repr. 会被视为整个输入序列的repr.
- [SEP]：有两个句子的文本会被串接成一个输入序列，并在两句之间插入这个token 以做区隔
- [UNK]：没出现在BERT 字典里头的字会被这个token 取代
- [PAD]：zero padding 遮罩，将长度不一的输入序列补齐方便做batch 运算
- [MASK]：未知遮罩，仅在预训练阶段会用到

In [1]:
vocab = tokenizer.vocab
print("vocab size: ", len(vocab))

NameError: name 'tokenizer' is not defined

In [17]:
text = "[CLS] 等到潮水 [MASK] 了，就知道谁沒穿裤子。"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(text)
print(tokens[:10], '...')
print(ids[:10], '...')


[CLS] 等到潮水 [MASK] 了，就知道谁沒穿裤子。
['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知'] ...
[101, 5023, 1168, 4060, 3717, 103, 749, 8024, 2218, 4761] ...


In [20]:
ids

[101,
 5023,
 1168,
 4060,
 3717,
 103,
 749,
 8024,
 2218,
 4761,
 6887,
 6443,
 3760,
 4959,
 6175,
 2094,
 511]

In [23]:
from transformers import BertForMaskedLM
# 除了 tokens 以外我們還需要辨別句子的 segment ids
tokens_tensor = torch.tensor([ids])  # (1, seq_len)
segments_tensors = torch.zeros_like(tokens_tensor)  # (1, seq_len)
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)

# 使用 masked LM 估計 [MASK] 位置所代表的實際 token 
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(tokens_tensor, segments_tensors)
    predictions = outputs[0]
    # (1, seq_len, num_hidden_units)
del maskedLM_model

# 將 [MASK] 位置的機率分佈取 top k 最有可能的 tokens 出來
masked_index = 5
k = 3
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

# 顯示 top k 可能的字。一般我們就是取 top 1 当做预测值
print("輸入 tokens ：", tokens[:10], '...')
print('-' * 50)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    tokens[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), tokens[:10]), '...')


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


輸入 tokens ： ['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知'] ...
--------------------------------------------------
Top 1 (65%)：['[CLS]', '等', '到', '潮', '水', '来', '了', '，', '就', '知'] ...
Top 2 ( 4%)：['[CLS]', '等', '到', '潮', '水', '过', '了', '，', '就', '知'] ...
Top 3 ( 4%)：['[CLS]', '等', '到', '潮', '水', '干', '了', '，', '就', '知'] ...


In [25]:
from transformers import AutoTokenizer, AutoModel
from bertviz import model_view

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese", output_attentions=True)
inputs = tokenizer.encode("[CLS] 等到潮水 [MASK] 了，就知道谁沒穿裤子。", return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 
model_view(attention, tokens)

ModuleNotFoundError: No module named 'bertviz'

In [7]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [8]:
inputs = tokenizer(['这是什么让我试试切词。你好', '这是什么让我试试切词。你好了卡就是打开了放假啊可是你的肌肤'], return_tensors="pt", padding=True)
outputs = model(**inputs)

In [9]:
inputs

{'input_ids': tensor([[ 101, 6821, 3221,  784,  720, 6375, 2769, 6407, 6407, 1147, 6404,  511,
          872, 1962,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0],
        [ 101, 6821, 3221,  784,  720, 6375, 2769, 6407, 6407, 1147, 6404,  511,
          872, 1962,  749, 1305, 2218, 3221, 2802, 2458,  749, 3123,  969, 1557,
         1377, 3221,  872, 4638, 5491, 5502,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}

In [10]:
outputs

MaskedLMOutput(loss=None, logits=tensor([[[ -7.9119,  -7.8119,  -7.8056,  ...,  -6.5529,  -6.1713,  -6.4862],
         [ -8.2322,  -8.0884,  -8.0818,  ...,  -6.7162,  -6.2229,  -6.5252],
         [-16.1794, -16.3523, -18.0674,  ...,  -8.4667,  -4.2078,  -5.2893],
         ...,
         [ -7.7972,  -7.5529,  -7.6924,  ...,  -4.1089,  -2.6128,  -3.1202],
         [ -7.9204,  -7.5319,  -7.7652,  ...,  -4.1665,  -2.4351,  -3.1046],
         [ -7.9449,  -7.6439,  -7.8875,  ...,  -4.2426,  -2.6454,  -3.0096]],

        [[ -8.0285,  -7.9644,  -7.9966,  ...,  -6.9183,  -6.6847,  -7.0758],
         [ -8.1193,  -7.9995,  -7.9447,  ...,  -6.7129,  -6.3368,  -6.5415],
         [-16.6799, -16.4066, -18.4110,  ...,  -9.9241,  -4.0629,  -5.6410],
         ...,
         [-13.4532, -13.5211, -12.6752,  ...,  -9.8302,  -4.2376, -10.3354],
         [-10.6056, -10.2510, -10.3465,  ...,  -8.8303,  -3.1233,  -7.3241],
         [-10.3576, -10.5163, -10.6644,  ...,  -8.2772,  -5.6076,  -8.5861]]],
       grad

In [11]:
outputs.logits.shape

torch.Size([2, 31, 21128])

In [13]:
model.get_output_embeddings()

Linear(in_features=768, out_features=21128, bias=True)

# Okay not let's figure out how to use the outputs

Sources: 

1. done https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1
2. https://towardsdatascience.com/bert-text-classification-in-a-different-language-6af54930f9cb
3. https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
4. https://towardsdatascience.com/how-bert-determines-search-relevance-2a67a1575ac4
5. https://bergum.medium.com/how-not-to-use-bert-for-search-ranking-4586716428d9


# Now try to do some document ranking.

Emm but no idea how to do that... No worries let's read some articles

1. https://medium.com/@papai143/information-retrieval-with-document-re-ranking-with-bert-and-bm25-7c29d738df73
2. https://medium.com/nerd-for-tech/bert-qe-contextualized-query-expansion-for-document-re-ranking-4f0f421840b9

# Try sentence transformer?

In [95]:
from sentence_transformers import SentenceTransformer
from pprint import pprint
import torch
from sentence_transformers import util

In [98]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda:0")  
    print("Running on the GPU")
else:
    DEVICE = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [99]:
# sentence_transformer_model = SentenceTransformer('distiluse-base-multilingual-cased-v1', device=DEVICE)
# later use this bigger model for higher performance 'paraphrase-multilingual-mpnet-base-v2'
sentence_transformer_model_v2 = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device=DEVICE)

In [100]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.',
    'Le renard brun et rapide saute par-dessus le chien paresseux.',
    '敏捷的棕色狐狸跳过懒惰的狗',
    'London is the best place on earth.',
    'I love London.']

In [101]:
sentence_embeddings = sentence_transformer_model_v2.encode(sentences)

In [102]:
sentence_embeddings

array([[-0.00925649, -0.25025994, -0.01117302, ...,  0.13195269,
        -0.16931531, -0.17440246],
       [ 0.15494236, -0.06809119, -0.01351547, ...,  0.01244846,
        -0.11615185, -0.12480734],
       [-0.09213712,  0.08014815, -0.00743247, ...,  0.15812053,
         0.255048  ,  0.03574583],
       ...,
       [-0.02064275,  0.12411997, -0.01361182, ...,  0.15744969,
         0.1656531 ,  0.00059244],
       [-0.11416887, -0.03790707, -0.01347593, ..., -0.00388702,
         0.07200608, -0.02660454],
       [-0.23245244,  0.07675917, -0.01163195, ...,  0.07803821,
         0.03771931, -0.18306209]], dtype=float32)

In [103]:
sentence_embeddings[0]

array([-9.25648678e-03, -2.50259936e-01, -1.11730229e-02,  1.17274500e-01,
        1.07081749e-01,  5.40037313e-03,  1.46492511e-01,  1.44696152e-02,
        1.06310651e-01,  6.33549914e-02,  2.63547003e-02,  2.65325576e-01,
       -7.31476173e-02,  1.89226300e-01,  1.05784861e-02, -2.06184015e-01,
        1.31849304e-01, -4.92107384e-02, -4.50022817e-02, -2.53913971e-03,
        7.82639310e-02, -2.97641549e-02,  4.54539713e-03, -1.12729985e-02,
       -1.20644532e-01,  2.43622176e-02, -1.43141404e-01,  2.72789598e-02,
       -1.48834633e-02,  3.08458321e-02, -2.59506553e-02,  1.07542817e-02,
       -8.04948136e-02, -1.60233527e-01,  6.63666204e-02,  2.12231167e-02,
       -1.53289288e-01,  3.46428272e-03, -1.33140400e-01, -6.58026412e-02,
       -1.31101459e-01, -8.27819705e-02,  5.41608520e-02,  2.45997291e-02,
       -1.14434473e-01,  4.85093007e-03,  1.80670828e-01,  1.63245164e-02,
        2.29559660e-01,  3.58686928e-04, -1.00527301e-01,  2.20974442e-02,
       -1.46725690e-02,  

In [105]:
print('sentences: ', sentences[0], '\n', sentences[4])
print('dot similarity: ', util.dot_score(sentence_embeddings[0], sentence_embeddings[4]))
print('cos similarity: ', util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[4]))

print('sentences: ', sentences[2], '\n', sentences[4])
print('dot similarity: ', util.dot_score(sentence_embeddings[2], sentence_embeddings[4]))
print('cos similarity: ', util.pytorch_cos_sim(sentence_embeddings[2], sentence_embeddings[4]))

print('sentences: ', sentences[3], '\n', sentences[4])
print('dot similarity: ', util.dot_score(sentence_embeddings[3], sentence_embeddings[4]))
print('cos similarity: ', util.pytorch_cos_sim(sentence_embeddings[3], sentence_embeddings[4]))

print('sentences: ', sentences[5], '\n', sentences[6])
print('dot similarity: ', util.dot_score(sentence_embeddings[5], sentence_embeddings[6]))
print('cos similarity: ', util.pytorch_cos_sim(sentence_embeddings[5], sentence_embeddings[6]))

sentences:  This framework generates embeddings for each input sentence 
 敏捷的棕色狐狸跳过懒惰的狗
dot similarity:  tensor([[0.5263]])
cos similarity:  tensor([[0.0797]])
sentences:  The quick brown fox jumps over the lazy dog. 
 敏捷的棕色狐狸跳过懒惰的狗
dot similarity:  tensor([[6.1115]])
cos similarity:  tensor([[0.9069]])
sentences:  Le renard brun et rapide saute par-dessus le chien paresseux. 
 敏捷的棕色狐狸跳过懒惰的狗
dot similarity:  tensor([[5.9588]])
cos similarity:  tensor([[0.9261]])
sentences:  London is the best place on earth. 
 I love London.
dot similarity:  tensor([[8.4054]])
cos similarity:  tensor([[0.7646]])


# okay now get some passages out of ES to test this

In [131]:
test_query = '班上同学欠钱不还怎么办'
INDEX = "efaqa-70" # index to search e.g. "msmacro-full"
FIELDS = ["passage"] # fields to search e.g. ["passage", "query"]

In [132]:
from es_helper import *

In [133]:
result_count, es_hits = es_search(test_query, cutoff = 10, index=INDEX, fields = FIELDS)
es_results = direct_es_search_result('dummy', test_query, es_hits)

/Users/leonlu-m1/opt/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [134]:
def rerank(es_results):
    '''
    gets a result table, reranks it, and returns a reranked result table
    '''
    passages = [hit['passage'] for hit in es_results.table]
    queries = [es_results.query_input]
    query_embeddings = sentence_transformer_model_v2.encode(queries, convert_to_tensor=True)
    sentence_embeddings = sentence_transformer_model_v2.encode(passages, convert_to_tensor=True)

    sentence_embeddings = sentence_embeddings.to(DEVICE)
    sentence_embeddings = util.normalize_embeddings(sentence_embeddings)

    query_embeddings = query_embeddings.to(DEVICE)
    query_embeddings = util.normalize_embeddings(query_embeddings)

    reranked_ranking = util.semantic_search(query_embeddings, sentence_embeddings, score_function=util.dot_score)

    reranked_table = []
    for index, entry_at_rank in enumerate(reranked_ranking[0]):
        corpus_id = entry_at_rank['corpus_id']
        entry = es_results.table[corpus_id]
        entry['score'] = entry_at_rank['score']
        entry['rank'] = index

        reranked_table.append(entry)
    es_results.table = reranked_table
    
    return es_results
        
es_results = rerank(es_results)

In [ ]:
es_results.table

[{'rank': 0,
  'qid': 'q0',
  'pid': 'p0',
  'query_label': '念书是为了坐办公室吗？？？为什么？为什么嗯念书是为了坐办公室吗唉，考试考得不好家里人说，看不上好大学很丢人，经常拿我和村里的好学生比较，受不了了但是家里人…………谢啦',
  'passage': '念书是为了自我提升，自我完善学习不是为了攀比，也不是为了单纯的为了做办公室我也是呢，父母很关心学习所以才进行比较的。不是办公室的问题，请你想想，在提升自己的同时，你的未来也会一片明亮，现在的学业成绩决定这未来伴侣的颜值，嘿嘿，放松心态，加油，送你一句话吧 不要让你野心配不上自己的实力，而你是否配的上你想要的生活呢？学生党路过感慨',
  'score': 0.5146608948707581},
 {'rank': 1,
  'qid': 'q0',
  'pid': 'p0',
  'query_label': '医师您好，我儿子（14岁）已经离家出走两天了。但是我发短信他还回，说让我们不要找他，他要去找工作，他在朋友家住，他对学习没有信趣。我们该怎么办？？两三年了吧不用报案吗我也这样想，但怕他出去被骗去做坏事他没让我知道在那个朋友那他出去后我能联系上他时我就是这么说的一般这样多久算安全的这样啊这个是一定的，只要他回来后我一定多咨询你们好的已经关注过了他的学习我让为我们也没强迫太多从没要求过他只是让他多努点，只要求中等水平可是只要中等水平不过吧，我们是在小城市的这也是，我们真的不知道该怎么办了',
  'passage': '孩子的今天是昨天父母教育的结果，你们以前是怎么面对和教育孩子的呢？不论如何给孩子找回来！不念书可以，离家出走怎么是什么意思！回家后说清楚彼此的心里话再去干自己干的事！否则以后还会出现别的突发事件！不能惯！没有规矩早晚出事……',
  'score': 0.5065162777900696},
 {'rank': 2,
  'qid': 'q0',
  'pid': 'p0',
  'query_label': '严重的拖延症感觉让自己生活很糟糕，而且在学习工作中也不能做到专注，总是一边看剧一边工作。持续时间很久了，现在会有对生活的困惑感和很深的挫败感。也没有允许的问题。。就这样肯定会效率低下 会事倍功半 现在就感觉自己干啥啥不